In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [3]:
from ICARUS.Database import DB
from ICARUS.Database.Database_3D import Database_3D
from time import sleep

INFO:root:Adding Vehicle at bmark
DEBUG:root:Entering AVL
INFO:root:Loading AVL Forces from C:\Users\tryfo\Desktop\Software\Icarus\Data\3D\bmark\forces.avl
DEBUG:root:Entering GenuVP3
DEBUG:root:Entering LSPT
INFO:root:Loading Forces from C:\Users\tryfo\Desktop\Software\Icarus\Data\3D\bmark\forces.lspt
INFO:root:Adding Vehicle at hermes
DEBUG:root:Entering AVL
INFO:root:Loading AVL Forces from C:\Users\tryfo\Desktop\Software\Icarus\Data\3D\hermes\forces.avl
DEBUG:root:Entering GenuVP3


In [4]:
DB.load_data()

INFO:root:Adding Vehicle at bmark
DEBUG:root:Entering AVL
INFO:root:Loading AVL Forces from C:\Users\tryfo\Desktop\Software\Icarus\Data\3D\bmark\forces.avl
DEBUG:root:Entering GenuVP3
DEBUG:root:Entering LSPT
INFO:root:Loading Forces from C:\Users\tryfo\Desktop\Software\Icarus\Data\3D\bmark\forces.lspt
INFO:root:Adding Vehicle at hermes
DEBUG:root:Entering AVL
INFO:root:Loading AVL Forces from C:\Users\tryfo\Desktop\Software\Icarus\Data\3D\hermes\forces.avl
DEBUG:root:Entering GenuVP3


# Open Database

In [5]:
db3d: Database_3D = DB.vehicles_db

In [6]:
db3d.forces['bmark'].keys()

Index(['AoA', 'AVL Fz', 'AVL Fx', 'AVL My', 'GenuVP3 Potential Fx',
       'GenuVP3 Potential Fy', 'GenuVP3 Potential Fz', 'GenuVP3 Potential Mx',
       'GenuVP3 Potential My', 'GenuVP3 Potential Mz', 'GenuVP3 2D Fx',
       'GenuVP3 2D Fy', 'GenuVP3 2D Fz', 'GenuVP3 2D Mx', 'GenuVP3 2D My',
       'GenuVP3 2D Mz', 'GenuVP3 ONERA Fx', 'GenuVP3 ONERA Fy',
       'GenuVP3 ONERA Fz', 'GenuVP3 ONERA Mx', 'GenuVP3 ONERA My',
       'GenuVP3 ONERA Mz', 'Fx', 'Fy', 'Fz', 'Mx', 'My', 'Mz',
       'LSPT Potential Fz', 'LSPT Potential Fx', 'LSPT Potential My',
       'LSPT 2D Fz', 'LSPT 2D Fx', 'LSPT 2D My'],
      dtype='object')

### Select Plane

In [7]:
planenames: list[str] = db3d.get_planenames()
for i, planename in enumerate(planenames):
    print(f"{i} : `{planename}") 
sleep(1)

indexes = [1]# range(0,len(planenames))
print(len(planenames))
plane = db3d.planes[planenames[indexes[0]]]
# dynplane = db3d.dyn_planes[planenames[index]]
planenames = [planenames[i] for i in indexes]

0 : `bmark
1 : `hermes
2


# Add XFLR Polars

In [8]:
from ICARUS.Computation.Solvers.XFLR5.polars import read_polars_3d
from ICARUS.Database import EXTERNAL_DB
for name in planenames:
    if name.startswith("XFLR"):
        continue

    if f"XFLR_{name}" not in planenames: 
        try:
            import os
            XFLR5PLANEDIR: str = os.path.join(EXTERNAL_DB,f"{name}.txt")
            read_polars_3d(XFLR5PLANEDIR, name)
            print(f"Imported XFLR polar for {name}")
            planenames.append(f"XFLR_{name}")
        except FileNotFoundError:
            pass
            # print(f"No XFLR polar found for {name}")
        

Imported XFLR polar for hermes


C:\Users\tryfo\Desktop\Software\Icarus\ICARUS\Computation\Solvers\XFLR5\polars.py:175: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df: DataFrame = pd.read_csv(


In [9]:
# planenames =[planenames[0]]
# planenames[0] = "Embraer_E190_Cruise 2D"
# db3d.data[planenames[0]] = db3d.data["XFLR_e190_cruise"]

In [10]:
# try:
#     XFLR5PLANEDIR = os.path.join(EXTERNAL_DB,f"e190_takeoff.txt")
#     read_polars_3d(db3d, XFLR5PLANEDIR, 'XFLR5_e190_takeoff')
#     planenames.append('Embraer_E190_takeoff')
#     planenames[-1]= "Embraer_E190_takeoff"
# except FileNotFoundError:
#     print("No XFLR polar found for e190_takeoff.txt")

In [11]:
# db3d.data[planenames[1]] = db3d.data["XFLR_Embraer_E190_takeoff"]

# Airplane Visualization

In [12]:
# plane.visualize()

# Polars

In [13]:
from ICARUS.Visualization.airplane.db_polars import plot_airplane_polars 
solvers = [
            # "GenuVP3 Potential",
            # "GenuVP3 2D", 
            # "GNVP3 ONERA",
            # "GNVP7 Potential",
            # "GNVP7 2D",
            # "LSPT Potential",
            # "LSPT 2D",
            "AVL"
        ]
axs, fig = plot_airplane_polars(
    planenames,
    solvers,
    plots = [["AoA", "CL"], ["AoA", "CD"], ["AoA", "Cm"]],
    size=(6,7),
)

DEBUG:matplotlib.font_manager:findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\tryfo\\anaconda3\\envs\\aero\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXSizThreeSymBol.ttf', name='STIXSizeThreeSym', style='normal', variant='normal', weight=700, stretch='normal', size='scalable')) = 10.335
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\tryfo\\anaconda3\\envs\\aero\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXSizThreeSymReg.ttf', name='STIXSizeThreeSym', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\tryfo\\anaconda3\\envs\\aero\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\STIXNonUniIta.ttf', name='STIXNonUnicode', style='italic', variant='normal', weight=400, stretch='normal', size='sca

In [14]:
# from ICARUS.Visualization.airplane.cg_investigation import cg_investigation

# cg_investigation(
#     planenames[0],
#     solvers,
#     size=(10,7),
# )

# Case Convergence

In [15]:
from ICARUS.Visualization.airplane.gnvp_convergence import plot_convergence

In [ ]:
angles2show = [0.]# All
plot_convergence(
    plane.name,
    angles = angles2show,
    solvers= ["All"],
    plot_error = False,
    size=(16,7)
)

DEBUG:matplotlib.ticker:vmin 0.4873890261709473 vmax 8.422031124761649
DEBUG:matplotlib.ticker:ticklocs array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])
DEBUG:matplotlib.ticker:vmin 0.4873890261709473 vmax 8.422031124761649
DEBUG:matplotlib.ticker:ticklocs array([2.e-02, 3.e-02, 4.e-02, 5.e-02, 6.e-02, 7.e-02, 8.e-02, 9.e-02,
       2.e-01, 3.e-01, 4.e-01, 5.e-01, 6.e-01, 7.e-01, 8.e-01, 9.e-01,
       2.e+00, 3.e+00, 4.e+00, 5.e+00, 6.e+00, 7.e+00, 8.e+00, 9.e+00,
       2.e+01, 3.e+01, 4.e+01, 5.e+01, 6.e+01, 7.e+01, 8.e+01, 9.e+01,
       2.e+02, 3.e+02, 4.e+02, 5.e+02, 6.e+02, 7.e+02, 8.e+02, 9.e+02])
DEBUG:matplotlib.ticker:vmin 0.4873890261709473 vmax 8.422031124761649
DEBUG:matplotlib.ticker:ticklocs array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])
DEBUG:matplotlib.ticker:vmin 0.4873890261709473 vmax 8.422031124761649
DEBUG:matplotlib.ticker:ticklocs array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])
DEBUG:matplotlib.ticker:vmin 0.4873890261709473 vmax 8.422031124761649
DEBUG:matplot

# Wake Visualization

In [ ]:
from ICARUS.Visualization.airplane.gnvp_wake import plot_gnvp_wake
from ICARUS.Database.utils  import angle_to_case

In [ ]:
from ICARUS.Database import DB3D
plane.CASEDIR =  os.path.join(DB3D, "Hermes")

In [ ]:
CASE: str = angle_to_case(0.)
plane = db3d.planes[plane.name]
# plane = db3d.planes['bmark']
# plane.visualize()
plot_gnvp_wake(3,plane,CASE, scale = "")

 # Loads Visualization

In [ ]:
from ICARUS.Visualization.airplane.gnvp_strips import gnvp_strips_2d, gnvp_strips_3d
NBs = [i+1 for i in range(len(plane.surfaces))]
print(plane)
print("NBs to Visualize: ",NBs)

stripData = gnvp_strips_2d(plane,CASE, NBs[0], category= 'Wind')

Plane Object: hermes
NBs to Visualize:  [1, 2, 3]


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\tryfo\\Desktop\\Software\\Icarus\\Data\\3D\\hermes\\0.00000_AoA'

# Dynamics

In [ ]:
states = db3d.states['plane_1']
for i,name in enumerate(states):
    print(f"{i}: {name}")

In [ ]:
db3d.states['plane_1']

In [ ]:
# name = 'atlas'
from ICARUS.Flight_Dynamics.state import State


state: State = states['plane_1']
state.stability_fd()
print(state)
state.plot_eigenvalues()
#   Longitudinal derivatives
#   Xu=    -0.16379         Cxu=   -0.049477
#   Xw=      1.1494         Cxa=     0.34723
#   Zu=     -3.0117         Czu=  -0.0035415
#   Zw=     -19.528         CLa=      5.8991
#   Zq=     -1.8321         CLq=      9.1645
#   Mu= -0.00060463         Cmu=  -0.0015122
#   Mw=    -0.45664         Cma=     -1.1421
#   Mq=    -0.39098         Cmq=     -16.193

#   Lateral derivatives
#   Yv=     -0.5496         CYb=    -0.16602
#   Yp=   -0.017022         CYp=  -0.0045504
#   Yr=     0.32383         CYr=    0.086568
#   Lv=    0.039731         Clb=   0.0053106
#   Lp=     -5.6227         Clp=     -0.6651
#   Lr=      1.0377         Clr=     0.12275
#   Nv=     0.28879         Cnb=    0.038602
#   Np=    -0.57586         Cnp=   -0.068117
#   Nr=     -0.1259         Cnr=   -0.014893


In [ ]:
state.state_space

# Sensitivity Plot

In [ ]:
from ICARUS.Visualization.airplane.gnvp_sensitivity import plot_sensitivity 